In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import sys

sys.path.append("../technique")
from utils import *

In [ ]:
def get(file):
    sheet_name = "2022-2023"
    return pd.read_excel(
        file,
        sheet_name=sheet_name,
    )

In [ ]:
df = get_data(
    "/home/thomas/Nextcloud/CodeursEnLiberte/EMS/dee/DEE_20230719_Données activités QF.xlsx",
    get,
)

In [ ]:
df.Activite.unique()

In [ ]:
df.UNITE.unique()

In [ ]:
df["SERVICE"] = df.Activite + "_" + df.UNITE

In [ ]:
len(df.SERVICE.unique())

In [ ]:
df[["SERVICE", "NOMBRE"]].groupby(by="SERVICE").sum().sort_values(
    "NOMBRE", ascending=False
)

In [ ]:
df["SERVICE2"] = df.Activite.str.replace("AL .*", "AL", regex=True) + "_" + df.UNITE

In [ ]:
df[["SERVICE2", "NOMBRE"]].groupby(by="SERVICE2").sum().sort_values(
    "NOMBRE", ascending=False
)

In [ ]:
usage = (
    df[["SERVICE", "N° PER", "MOIS", "NOMBRE"]]
    .groupby(by=["N° PER", "MOIS", "SERVICE"])
    .sum()
)  # .reset_index()

In [ ]:
usage

In [ ]:
flat_usage = usage.reset_index()
flat_usage

In [ ]:
multiple = (
    flat_usage[["N° PER", "MOIS", "SERVICE"]].groupby(by=["N° PER", "MOIS"]).count()
)
multiple[multiple.SERVICE > 1].index

In [ ]:
data = flat_usage.join(
    multiple[multiple.SERVICE > 1], on=["N° PER", "MOIS"], rsuffix="r", how="inner"
).set_index(["N° PER", "MOIS"])

In [ ]:
data

In [ ]:
with pd.option_context("display.max_rows", 300):
    display(data[0:100])

In [ ]:
data[data.SERVICE.str.contains("APM")][["SERVICE", "NOMBRE"]].groupby("SERVICE").count()

In [ ]:
data[data.SERVICE.str.contains("APM_M")].index.values

In [ ]:
data.loc[(217881, pd.Timestamp("2022-10-01 00:00:00"))]

In [ ]:
with pd.option_context("display.max_rows", 800):
    display(data.loc[data[data.SERVICE.str.contains("APM_M")].index.values])

In [ ]:
# Il semblerait que les données soient de la fréquentation et donc
# qu'il faille faire le lien avec les services/tarifications

In [ ]:
df

In [ ]:
df_apm = df[df.Activite == "APM"].pivot(
    index=["N° FAM", "QF", "N° PER", "MOIS"], columns="UNITE", values="NOMBRE"
)

In [ ]:
df_apm[(~df_apm.M.isna()) * (df_apm.S == 0)]

In [ ]:
df_apm[df_apm.S == 0]

In [ ]:
df_apm_base = df_apm.reset_index()

In [ ]:
df_apm_base

In [ ]:
df_apm_base["USE"] = True

In [ ]:
df_apm_mensuel = (
    df_apm_base[["N° FAM", "QF", "N° PER", "MOIS", "USE"]]
    .groupby(["N° FAM", "QF", "N° PER", "MOIS"])
    .count()
)

In [ ]:
df_apm_usage = (
    df_apm_mensuel.reset_index()[["N° FAM", "QF", "N° PER", "MOIS"]]
    .groupby(["N° FAM", "QF", "N° PER"])
    .count()
    .reset_index()
)

In [ ]:
df_apm_usage

In [ ]:
df_apm_usage

In [ ]:
from openfisca_survey_manager.scenarios import AbstractSurveyScenario
from openfisca_france import CountryTaxBenefitSystem
from openfisca_france.model.base import Famille, FoyerFiscal, Menage
from openfisca_core import periods

In [ ]:
base = CountryTaxBenefitSystem()
base.load_extension("openfisca_france_local")

In [ ]:
df_apm_usage

In [ ]:
def build_data(df):
    individu_df = pd.DataFrame(
        {
            "famille_id": list(range(len(df))),
        }
    )
    famille_df = pd.DataFrame(
        {
            "qfrule": "QF==" + df.QF.astype("str"),
        }
    )
    determine_qf(famille_df, qfrules_constant)
    menage_df = pd.DataFrame({})
    foyerfiscaux_df = pd.DataFrame({})

    individu_df["famille_role_index"] = 0
    individu_df["foyer_fiscal_id"] = individu_df.famille_id
    individu_df["foyer_fiscal_role_index"] = 0
    individu_df["menage_id"] = individu_df.famille_id
    individu_df["menage_role_index"] = 0

    return dict(
        input_data_frame_by_entity=dict(
            individu=individu_df,
            famille=famille_df,
            menage=menage_df,
            foyer_fiscal=foyerfiscaux_df,
        )
    )

In [ ]:
data = build_data(df_apm_usage)
scenario = StrasbourgSurveyScenario(base, data=data)

In [ ]:
prix = scenario.simulation.calculate(
    "strasbourg_prix_periscolaire_maternelle", base_period
)
prix

In [ ]:
(prix * df_apm_usage.MOIS).sum()

In [ ]:
df.SERVICE2.unique()

In [ ]:
fields = {
    "AL_J": "strasbourg_prix_periscolaire_loisirs_journee",
    "AL_1/2J": "strasbourg_prix_periscolaire_loisirs_demi_journee",
    "AL_R": "strasbourg_prix_periscolaire_loisirs_repas",
    "AL_P": "strasbourg_prix_periscolaire_loisirs_panier",
}

In [ ]:
df_al = (
    df[df.SERVICE2.str.contains("AL_")][
        ["N° FAM", "QF", "N° PER", "SERVICE2", "NOMBRE"]
    ]
    .groupby(["N° FAM", "QF", "N° PER", "SERVICE2"])
    .sum()
    .reset_index()
)

In [ ]:
df_al

In [ ]:
df_al.SERVICE2.unique()

In [ ]:
data_al = build_data(df_al)

In [ ]:
scenario_al = StrasbourgSurveyScenario(base, data=data_al)

In [ ]:
res_al = pd.DataFrame(
    data={v: scenario_al.simulation.calculate(fields[v], base_period) for v in fields}
)
res_al

In [ ]:
df_al

In [ ]:
idx, cols = pd.factorize(df_al.SERVICE2)
idx, cols

In [ ]:
idx.shape

In [ ]:
res_al.shape

In [ ]:
df_al["PRIX"] = res_al.reindex(cols, axis=1).to_numpy()[np.arange(len(res_al)), idx]
df_al

In [ ]:
df_al["COUT"] = df_al.PRIX * df_al.NOMBRE

In [ ]:
df_al["COUT"].sum()

In [ ]:
df_al[["COUT", "SERVICE2"]].groupby("SERVICE2").sum()

In [ ]:
df_al[["PRIX", "SERVICE2"]].groupby("SERVICE2").hist(bins=20)